# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os

import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig

from azureml.core import Dataset

In [2]:
print(azureml.core.VERSION)

1.33.0


## Dataset

Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:

ws = Workspace.from_config()
ws.write_config(".azureml")

experiment_name = 'udacity-capstone-project'

experiments = Experiment.list(ws, experiment_name=experiment_name)
if not experiments:
      experiment = Experiment(workspace=ws, name=experiment_name)
else:
      experiment = experiments[0]


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-157230
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-157230


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster or reuse it if it is available.
cpu_cluster_name = "cpu-cluster-01vx"
vm_size = "Standard_D3_V2"
min_nodes = 0
max_nodes = 6

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster. use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           min_nodes=min_nodes,
                                                           max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# a detailed status for the current cluster.
print(compute_target.get_status().serialize())

Found existing cluster. use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 5, 'targetNodeCount': 5, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 5, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-09-07T14:06:30.700000+00:00', 'errors': None, 'creationTime': '2021-09-07T13:56:42.947586+00:00', 'modifiedTime': '2021-09-07T13:57:08.535357+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


## Hyperdrive Configuration
 
Logistic regression was used to build the classification model. 
Two hyperparameters of the inverse of regularization (C) and the maximum number of iterations (max-iter) were chosen to run HyperDrive.
C is chosen to control the effective size of parameters.
max-iter was tested to determine how long the training should be done to find an optimal model.
For each run, the two parameters were randomly selected (RandomParameter Sampling).
For the early stopping policy, we chose BanditPolicy that seems to be efficient of choosing the early stopping point.

In [5]:
import os
import shutil

from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

In [6]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.2, evaluation_interval=10, delay_evaluation=5) 

# Create the different params that you will be using during training
parameter_space = { "--C": uniform(0.1, 1.0), 
                    "--max_iter": choice([50, 100, 500, 1000]) }
param_sampling = RandomParameterSampling(parameter_space)



In [7]:
script_folder = "training"
os.makedirs(os.path.join(".", script_folder), exist_ok=True)

# Create a SKLearn estimator for use with train.py
shutil.copy("./train.py", os.path.join(".", script_folder))
source_directory=os.path.join(".", script_folder)

In [8]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- numpy
- pandas
- scikit-learn
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [9]:
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [10]:
# Use ScrptRunConfig
src = ScriptRunConfig(source_directory=source_directory,
                      script='train.py',
                      arguments=[],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig
hyperdrive_run_config = HyperDriveConfig(
                        run_config=src,
                        hyperparameter_sampling=param_sampling,
                        policy=early_termination_policy,
                        primary_metric_name="accuracy",
                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                        max_total_runs=30,
                        max_concurrent_runs=max_nodes-1
                        ) 

In [11]:
# Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [13]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_b3dd1119-9945-4270-9050-89d5d68053f5
Web View: https://ml.azure.com/runs/HD_b3dd1119-9945-4270-9050-89d5d68053f5?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-157230/workspaces/quick-starts-ws-157230&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-09-07T14:45:35.841414][API][INFO]Experiment created<END>\n""<START>[2021-09-07T14:45:36.717438][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-09-07T14:45:37.128315][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_b3dd1119-9945-4270-9050-89d5d68053f5
Web View: https://ml.azure.com/runs/HD_b3dd1119-9945-4270-9050-89d5d68053f5?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-157230/workspaces/quick-starts-ws-157230&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_b3dd1119-9945-4270-9050-89d5d68053f5',
 'target': 'cpu-cluster-01vx',
 'status': 'Completed',
 'startTimeUtc': '2021-09-07T14:45:35.474921Z',
 'endTimeUtc': '2021-09-07T15:07:12.80574Z',
 'properties': {'primary_metric_config': '{"name": "accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0ec95335-b0fb-43c0-9e80-5bbf5edc9f53',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.7',
  'best_child_run_id': 'HD_b3dd1119-9945-4270-9050-89d5d68053f5_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg157230.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_b3dd1119-9945-4270-9050-89d5d68053f5/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr=b&sig=9hh7KSWJ

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()["runDefinition"]["arguments"]

In [15]:
print("Best Run Id      :", best_run.id)
print("Accuracy         :", best_run_metrics["accuracy"])
print("--C              :", parameter_values[1])
print("--max-iter       :", parameter_values[3])

print(parameter_values)

Best Run Id      : HD_b3dd1119-9945-4270-9050-89d5d68053f5_0
Accuracy         : 0.7
--C              : 0.95028673109488
--max-iter       : 1000
['--C', '0.95028673109488', '--max_iter', '1000']


In [16]:
best_run_filenames = best_run.get_file_names()
for filename in best_run_filenames:
    print(filename)

azureml-logs/20_image_build_log.txt
azureml-logs/55_azureml-execution-tvmps_71b47629891e1f38ed6673b41148e35b6f34fc5ef41e9117f7efa7749a52b181_d.txt
azureml-logs/65_job_prep-tvmps_71b47629891e1f38ed6673b41148e35b6f34fc5ef41e9117f7efa7749a52b181_d.txt
azureml-logs/70_driver_log.txt
azureml-logs/75_job_post-tvmps_71b47629891e1f38ed6673b41148e35b6f34fc5ef41e9117f7efa7749a52b181_d.txt
azureml-logs/process_info.json
azureml-logs/process_status.json
logs/azureml/100_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
logs/azureml/job_prep_azureml.log
logs/azureml/job_release_azureml.log
outputs/model_0.9503_1000.joblib


In [17]:
# Save the best model
# The last element of best_run_filenames is a joblib file.
best_run.download_file(best_run_filenames[-1], output_file_path=os.path.join(".", script_folder))